In [1]:
import pandas as pd
import numpy as np
import os
import sagemaker
import json

from sagemaker.tensorflow import TensorFlow

In [2]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

## Load constants and directories

In [3]:
constants = json.load( open( "helpers/objects/constants_dict.json", "r" ) )
tag2idx = json.load( open("helpers/objects/tag2idx.json", "r"))
data_directories = json.load( open("helpers/objects/data_directories.json", "r"))

In [4]:
print('Constants:')
display(constants)
print('Directories:')
display(data_directories)

Constants:


{'N_WORDS': 35178, 'N_TAGS': 17, 'MAX_LEN': 50}

Directories:


{'train_data_directory': 's3://sagemaker-eu-west-1-087816224558/named_entity_recognition/bilstm_data/bilstm_train.csv',
 'test_data_directory': 's3://sagemaker-eu-west-1-087816224558/named_entity_recognition/bilstm_data/bilstm_test.csv'}

## Model using GPU instance

In [3]:
! pygmentize models/train_bilstm.py

import argparse, os
import numpy as np
import pandas as pd
import json

from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Embedding,
    SpatialDropout1D,
    Bidirectional,
    LSTM,
    TimeDistributed,
    Dense
)

class BiLSTM(Model):

    def __init__(self):
        super(BiLSTM, self).__init__()
        self.embedding = Embedding(input_dim=N_WORDS, output_dim=50, input_length=MAX_LEN)
        self.spatial_dropout = SpatialDropout1D(0.1)
        self.bilstm = Bidirectional(LSTM(units=100, return_sequences=True))
        self.tddense = TimeDistributed(Dense(N_TAGS, activation="softmax"))

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.spatial_dropout(x)
        x = self.bilstm(x)
        return self.tddense(x)


if __name__ == '__main__':
        
    parser = argparse.ArgumentParser()

    parser.add_argument('--epochs', type=int, default=5)
    parser.add_argument('--batch-size', type=int, default=32)
    
    parser.add

### Define hyperparameters

In [25]:
hyperparameters={
    'epochs': 5,
    'batch-size': 64,
    'max-len': constants['MAX_LEN'],
    'n-tags': constants['N_TAGS'],
    'n-words': constants['N_WORDS'],
    'model-version': '1'
}

### Declare estiamtor and train the model

In [26]:
tf_estimator = TensorFlow(entry_point='train_bilstm.py', 
                          source_dir="models",
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.p2.xlarge',
                          framework_version='2.1.0', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters=hyperparameters
                         )

In [27]:
tf_estimator.fit({'training': data_directories['train_data_directory']})

2020-05-28 12:31:07 Starting - Starting the training job...
2020-05-28 12:31:10 Starting - Launching requested ML instances.........
2020-05-28 12:32:43 Starting - Preparing the instances for training......
2020-05-28 12:33:59 Downloading - Downloading input data...
2020-05-28 12:34:29 Training - Downloading the training image......
2020-05-28 12:35:35 Training - Training image download completed. Training in progress..2020-05-28 12:35:39,962 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-05-28 12:35:40,456 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "batch-size": 64,
        "max-len": 50,
        "n-words": 35178,
  

## Deploy model

In [28]:
tf_predictor = tf_estimator.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

-----------!

## Clean up

In [ ]:
# tf_predictor.delete_endpoint()

In [ ]:
# import boto3
# bucket_to_delete = boto3.resource('s3').Bucket(bucket)
# bucket_to_delete.objects.all().delete()